In [1]:
import os
from typing import List
from pydantic import BaseModel
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langgraph.graph import StateGraph, END

In [2]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
llm=init_chat_model("openai:gpt-4o")

In [4]:
docs = TextLoader("sample_docs.txt").load()
chunks = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).split_documents(docs)
vectorstore = FAISS.from_documents(chunks, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [6]:
# State Definition
class RAGReflectionState(BaseModel):
        question: str
        retrieved_docs: List[Document] = []
        answer: str = ""
        reflection: str = ""
        revised: bool = False
        attempts: int = 0

In [11]:
#  Nodes

def retrieve_docs(state: RAGReflectionState) -> RAGReflectionState:
        docs = retriever.invoke(state.question)
        return state.model_copy(update={"retrieved_docs": docs})
def generate_answer(state: RAGReflectionState) -> RAGReflectionState:
    context = "".join([doc.page_content for doc in state.retrieved_docs])
    prompt = f"""
    Use the following context to answer the question:\n",
    Context:
    {context}
    Question:
    {state.question}
    """
    answer = llm.invoke(prompt).content.strip()
    return state.model_copy(update={"answer": answer, "attempts": state.attempts + 1})

In [12]:
# Self-Reflect
def reflect_on_answer(state: RAGReflectionState) -> RAGReflectionState:
    prompt = f"""
    Reflect on the following answer to see if it fully addresses the question.
    State YES if it is complete and correct, or NO with an explanation.
    Question: {state.question}
    Answer: {state.answer}
    Respond like:
    Reflection: YES or NO
    Explanation: ..."
    """
    result = llm.invoke(prompt).content
    is_ok = "reflection: yes" in result.lower()
    return state.model_copy(update={"reflection": result, "revised": not is_ok})

In [14]:
# Finalizer
def finalize(state: RAGReflectionState) -> RAGReflectionState:
        return state

In [15]:
# LangGraph DAG\n",
builder = StateGraph(RAGReflectionState)
builder.add_node("retriever", retrieve_docs)
builder.add_node("responder", generate_answer)
builder.add_node("reflector", reflect_on_answer)
builder.add_node("done", finalize)
builder.set_entry_point("retriever")
builder.add_edge("retriever", "responder")
builder.add_edge("responder", "reflector")
builder.add_conditional_edges(
    "reflector",
    lambda s: "done" if not s.revised or s.attempts >= 2 else "retriever"
    )
builder.add_edge("done", END)
graph = builder.compile()

In [17]:
#Run the Agent
if __name__ == "__main__":
    user_query = "What are the transformer variants in production deployments?"
    init_state = RAGReflectionState(question=user_query)
    result = graph.invoke(init_state)
    print("🧠 Final Answer:", result["answer"])
    print("n🔁 Reflection Log:", result["reflection"])
    print("🔄 Total Attempts:", result["attempts"])

🧠 Final Answer: The question appears to be about transformer variants used in production deployments, but the provided context doesn't directly address this. However, common transformer variants that are often used in production for various applications include:

1. **BERT (Bidirectional Encoder Representations from Transformers)**: Widely used for tasks like natural language understanding and question answering.

2. **GPT (Generative Pre-trained Transformer)**: Used in tasks requiring text generation, such as chatbots and conversational AI.

3. **Transformers for Vision (e.g., Vision Transformer, ViT)**: Used in image classification and other computer vision tasks.

4. **T5 (Text-to-Text Transfer Transformer)**: Designed to handle a variety of NLP tasks through a unified text-to-text framework.

5. **DistilBERT**: A lighter and faster version of BERT, optimized for performance and efficiency while retaining much of BERT's accuracy.

These transformer models have been adapted and fine-